# 电商移动推荐算法竞赛实践


比赛入口

* https://tianchi.aliyun.com/competition/entrance/231522/introduction


本课件参考了以下链接中的想法，代码为重新编写
* https://blog.csdn.net/Datuqiqi/article/details/46834579
* https://blog.csdn.net/Snoopy_Yuan/article/category/6924508
* https://github.com/chenkkkk/TianChi_YiDongTuiJian_forecast 



本课件数据下载地址，此数据做了缩短，原来一个月的数据为了方便缩短为12-10到12-18日，如果需要全量数据请自行参加比赛下载，数据说明和之前的天猫品牌行为数据类似，behavior_type分为1浏览，2收藏，3购物车，4购买，多了经纬度数据user_geohash但是没用。
* https://pan.baidu.com/s/13SrbmDWGBqQDlnfqB5yI1w


重要代码

* me.accuracy_score(y_true, y_pred)
* me.recall_score(y_true, y_pred)
* me.f1_score(y_true, y_pred)
* me.precision_score(y_true, y_pred)

## 推荐的原理与特征处理

#### 数据读取

https://www.bilibili.com/video/BV1s7411w7Xv?p=17

#### groupby？

https://www.bilibili.com/video/BV1Z5411L7vU?p=9


#### unstack

https://blog.csdn.net/qq_42874547/article/details/89056000

#### 数据处理三板斧——map、apply、applymap详解

https://zhuanlan.zhihu.com/p/100064394

In [1]:
import sklearn.metrics as me
import numpy as np
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier #公认这里比较好的算法，我们就直接用了，而且这个方法不太需要特征标准化   
from sklearn.model_selection import train_test_split,GridSearchCV 

* 以每天的浏览，收藏，购物车，购买行为数量构建准特征

In [ ]:
u_i_table=pd.read_csv(r"F:\data\tianchi\tiancixinrentuijian\fresh_comp_offline\tianchi_user2.csv",parse_dates=[5])
i_table=pd.read_csv(r"F:\data\tianchi\tiancixinrentuijian\fresh_comp_offline\tianchi_fresh_comp_train_item.csv")
u_i_table = u_i_table[u_i_table.item_id.isin(i_table.item_id.values)]#过滤掉所有不属于预测商品集的商品行为记录

time                            2014-12-10          2014-12-11           \
behavior_type                            1  2  3  4          1  2  3  4   
user_id item_id   item_category                                           
492     31040941  6986                   0  0  0  0          0  0  0  0   
3726    264960    10412                  0  0  0  0          2  0  0  0   
19137   98921119  3942                   0  0  0  0          0  0  0  0   
38745   38770686  3942                   0  0  0  0          0  0  0  0   
        272985263 3942                   0  0  0  0          0  0  0  0   

time                            2014-12-12     ... 2014-12-16    2014-12-17  \
behavior_type                            1  2  ...          3  4          1   
user_id item_id   item_category                ...                            
492     31040941  6986                   0  0  ...          0  0          0   
3726    264960    10412                  0  0  ...          0  0          0   
19137   98921119  3942                   2  0  ...          0  0          0   
38745   38770686  3942                   0  0  ...          0  0          0   
        272985263 3942                   0  0  ...          0  0          0   

time                                     2014-12-18           
behavior_type                    2  3  4          1  2  3  4  
user_id item_id   item_category                               
492     31040941  6986           0  0  0          0  0  0  0  
3726    264960    10412          0  0  0          0  0  0  0  
19137   98921119  3942           0  0  0          0  0  0  0  
38745   38770686  3942           0  0  0          0  0  0  0  
        272985263 3942           0  0  0          0  0  0  0  

[5 rows x 36 columns]

In [8]:
len(u_i_table)

682059

## 基于单天行为简单统计的算法

* 基于上面的准特征矩阵构建可用的特征矩阵与目标向量，并通过标准流程完成训练，然后基于精准度对模型进行评价
* 注意这里以16和17的数据为训练集，以17和18的数据为测试集。

* GradientBoostingClassifier()
* map(lambda x:1 if x>0 else 0)
* me.accuracy_score(y_true, y_pred)

## 二分类的模型评价


|  预测与真实 | 真实为正 | 真实为负 |
| ------ | ------ | ------ |
|预测为正 | TP：True Positive,被判定为正样本，事实上也是正样本。 | FP：False Positive,被判定为正样本，但事实上是负样本。 |
| 预测为负| FN：False Negative,被判定为负样本，但事实上是正样本 | TN：True Negative,被判定为负样本，事实上也是负样本 |




$\text{precision} = \frac{tp}{tp + fp}$

* 查准率，所有正预测中正确缺的比率。

$\text{recall} = \frac{tp}{tp + fn}$

* 查全率，所有正样本中被预测到的比率。

$F_\beta = (1 + \beta^2) \frac{\text{precision} \times \text{recall}}{\beta^2 \text{precision} + \text{recall}}$

* F 值可以解释为 precision （精度）和 recall （召回）的加权调和平均值。 $F_\beta$ 值达到其最佳值 1 ，其最差分数为 0 。当 $\beta = 1$, F_\beta 和 F_1 是等价的， 此时意味着recall和 precision同样重要，# beta值越小，表示越看中precision，beta值越大，表示越看中recall，不同等重要的场景。

* 以上指标只关注正样本的预测情况。这和很多实际情况是一致的，比如推荐系统，欺诈识别等等

730 268137


## 样本不均衡问题

* 通过随机选择负样本达到正负均衡

* pd.concat

1  2  3  4  y
user_id item_id   item_category               
492     31040941  6986           0  0  0  0  0
3726    264960    10412          0  0  0  0  0
19137   98921119  3942           0  0  0  0  0
38745   38770686  3942           0  0  0  0  0
        272985263 3942           0  0  0  0  0

## 超参数——调参